# Creating Folkersma joint network layer

Assuming that the network data we received from Niras and Folkersma are in the folders `data/NIRAS` and `data/FOLKERSMA`, respectively (from Dropbox [here for Niras](https://www.dropbox.com/scl/fo/76ome1g3nqpr71akrifba/h?rlkey=3vijuusrgwil3wprd0gmfajhj&dl=0) and [here for Folkersma]())

This script imports nodes and edges from both sources and merges them (one file for all nodes; one file for all edges)

Run from main repo folder!

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# read in Niras data
edges1 = gpd.read_file("data/NIRAS/straekninger.gpkg")
nodes1 = gpd.read_file("data/NIRAS/knudepunkter.gpkg")

# read in Folkersma data
edges2 = gpd.read_file("data/FOLKERSMA/stretch.shp")
nodes2 = gpd.read_file("data/FOLKERSMA/node.shp")

In [3]:
# check that crs is the same for all data sets
assert all([edges1.crs, edges2.crs, nodes1.crs, nodes2.crs])
assert edges1.crs == edges2.crs
assert nodes1.crs == nodes2.crs
assert edges1.crs == nodes1.crs


In [4]:
# make sure we have the same columns present in both data sets
assert sorted(list(edges1.columns)) == sorted(list(edges2.columns))
assert sorted(list(nodes1.columns)) == sorted(list(nodes2.columns))

In [5]:
# make sure data sets don't intersect
assert edges1.unary_union.intersection(edges2.unary_union).is_empty
assert nodes1.unary_union.intersection(nodes2.unary_union).is_empty

In [6]:
# merge the two data frames
nodes = pd.concat([nodes1, nodes2], join = "outer", ignore_index=True)
edges = pd.concat([edges1, edges2], join = "outer", ignore_index=True)
assert len(nodes.columns) == len(nodes1.columns)
assert len(edges.columns) == len(edges1.columns)

In [8]:
# explode geometries
nodes = nodes.explode(ignore_index=True)
edges = edges.explode(ignore_index=True)

In [13]:
# save as gpkgs
nodes.to_file("data/raw/network/nodes.gpkg")
edges.to_file("data/raw/network/edges.gpkg")